### IAPPT
Este archivo contiene la creacion y entrenamiento del modelo que usamos para determinar si la mano esta en posicion de piedra, papel o tijera.
Antes de nada ejecutaremos los imports necesarios.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import clear_output
import tensorflow as tf
import common.constants as const

# Keras 
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

# Pandas para lectura csv
import pandas as pd


### Datasets
Primero hemos de entender que datos estamos pasando a nuestro modelo exactamente.
Mediante mediapie obtenemos una serie de puntos llamados landmarks, cada uno con un numero representandolo. 

![alt text](https://google.github.io/mediapipe/images/mobile/hand_landmarks.png)

Para obtener y guardar estos datos en el csv se encarga el ejecutable ```csv_generator.py```.
De todos estos landmarks solo nos interesan nueve: 6,8,10,12,14,16,18,20.
Y quedan definidos asi en el csv:

- gesto: que se representa si esa fila que gesto de PPT representa
- muneca_Y: coordenada Y del landmark 0
- muneca_X: coordenada X del landmark 0
- indicep_Y: coordenada Y del landmark 6
- indicep_X: coordenada X del landmark 6
- indicet_Y: coordenada Y del landmark 8
- indicet_Y: coordenada X del landmark 8
- ...

In [4]:
# A almacenamos los datos de los ficheros
data_train = pd.read_csv(const.TRAINING_CSV)
data_eval = pd.read_csv(const.EVALUATION_CSV)

Antes de usar el dataset eliminamos la columna que representa el resultado y la almacenamos en una variable



In [3]:
# Eliminamos el campo con el resultado y lo almacenamos
train_y = to_categorical(data_train.pop('gesto'))
eval_y = to_categorical(data_eval.pop('gesto'))

###  Creando el modelo
Usaremos un modelo secuencial con cuatro capas ```1 input```, ```2 ocultas``` y ```1 output``` con 
Los parametros son estos:

- ```500/100/50/3``` representa el la cantidad de outputs de esta capa, es decir el número de conexiones con la capa inferior.

- ```input_dim``` cantidad de parametros en la entrada.

- ```activation``` una funcion de activacion es la que transforma la señal de un neuron, este parametro especifica que tipo de funcion usamos en cada capa.

- ```optimizer``` los optimizadores son algoritmos usados para disminuir errores del modelo. ```learning_rate``` establece la frecuencia con la que el algoritmo actualiza sus parametros.
- ```loss``` funcion utilizada para cuantificar como de bueno es el desempeño del modelo cuanto mas cercano es a 0 mejor.

- ```metrics``` es el objetivo del modelo (precision en este caso). 

In [15]:
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=18))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compilar el modelo
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.03), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Mostrar estructura del modelo
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 500)               9500      
                                                                 
 dense_9 (Dense)             (None, 100)               50100     
                                                                 
 dense_10 (Dense)            (None, 50)                5050      
                                                                 
 dense_11 (Dense)            (None, 3)                 153       
                                                                 
Total params: 64,803
Trainable params: 64,803
Non-trainable params: 0
_________________________________________________________________


### Entrenar el modelo

Aqui es donde empieza a aprender y clasificar.
Para ello se llama con el metodo ```fit()``` con los siguientes parametros:

- ```data_train``` es el dataset con los landmarks.

- ```train_y``` contiene la columna con el dato de gesto para que el modelo compruebe si acierta o falla.

- ```batch_size``` cantidad de filas de data_train que se procesan antes de actualizar parametros del modelo.

- ```epochs``` numero de repeticiones sobre el dataset completo.

In [16]:
model.fit(data_train, train_y, batch_size=128, epochs=10)

Epoch 1/10
18/18 [==============================] - 1s 7ms/step - loss: 1.1687 - accuracy: 0.4385
Epoch 2/10
18/18 [==============================] - 0s 6ms/step - loss: 0.6273 - accuracy: 0.6999
Epoch 3/10
18/18 [==============================] - 0s 7ms/step - loss: 0.2837 - accuracy: 0.8888
Epoch 4/10
18/18 [==============================] - 0s 6ms/step - loss: 0.1292 - accuracy: 0.9521
Epoch 5/10
18/18 [==============================] - 0s 7ms/step - loss: 0.0743 - accuracy: 0.9750
Epoch 6/10
18/18 [==============================] - 0s 6ms/step - loss: 0.0424 - accuracy: 0.9864
Epoch 7/10
18/18 [==============================] - 0s 6ms/step - loss: 0.0317 - accuracy: 0.9912
Epoch 8/10
18/18 [==============================] - 0s 6ms/step - loss: 0.0415 - accuracy: 0.9859
Epoch 9/10
18/18 [==============================] - 0s 6ms/step - loss: 0.0519 - accuracy: 0.9807
Epoch 10/10
18/18 [==============================] - 0s 6ms/step - loss: 0.0389 - accuracy: 0.9890


### Resultados
Para visualizar la precision del modelo, ejecutamos esta celda.

In [17]:
scores = model.evaluate(data_eval, eval_y, verbose=False)
print(f'Precision del modelo: {scores[1]*100:.1f}%')  

Precision del modelo: 99.7%


### Guardar el modelo
Si estamos contentos con el rendimiento lo podemos guardar de esta manera:

In [ ]:
model.save('model/gesture_classifier.h5')

Ejecutando esta celda eliminaremos los modelos anteriores:

In [ ]:
import shutil
import os
try:
    shutil.rmtree('model')
except:
    print('Error eliminando carpeta model/')
os.mkdir('model')